## Leia o arquivo dos dados da Receita Federal e cruze eles com os dados da CEAP

### CEAP

#### Baixando os dados da CEAP

Todo aquele processo que já conhecemos :)

In [1]:
import requests

url = "http://www.camara.leg.br/cotas/Ano-2019.csv.zip"
r = requests.get(url)

In [2]:
file = open("Ano-2019.csv.zip", "wb")
file.write(r.content)
file.close()

In [3]:
from zipfile import ZipFile

zip_file = ZipFile("Ano-2019.csv.zip", 'r')

In [4]:
zip_file.extract(member="Ano-2019.csv", path="reembolso-2019")

'reembolso-2019\\Ano-2019.csv'

In [5]:
zip_file.close()

#### Lendo os dados da CEAP

In [6]:
import pandas as pd

# usamos essa configuração para o pandas mostrar todas as colunas do dataframe.
pd.set_option('display.max_columns', None)

In [8]:
DTYPE = {
    'txNomeParlamentar': str,
    'ideCadastro': str,
    'nuCarteiraParlamentar': str,
    'nuLegislatura': str,
    'sgUF': str,
    'sgPartido': str,
    'codLegislatura': str,
    'numSubCota': str,
    'txtDescricao': str,
    'numEspecificacaoSubCota': str,
    'txtDescricaoEspecificacao': str,
    'txtFornecedor': str,
    'txtCNPJCPF': str,
    'txtNumero': str,
    'indTipoDocumento': str,
    'datEmissao': str,
    'vlrDocumento': float,
    'vlrGlosa': str,
    'vlrLiquido': float,
    'numMes': str,
    'numAno': str,
    'numParcela': str,
    'txtPassageiro': str,
    'txtTrecho': str,
    'numLote': str,
    'numRessarcimento': str,
    'nuDeputadoId': str,
    'ideDocumento': str,
}

df_reembolso = pd.read_csv("reembolso-2019/Ano-2019.csv", sep=";", dtype=DTYPE)

df_reembolso["datEmissao"] = df_reembolso["datEmissao"].astype('datetime64[ns]')

In [9]:
df_reembolso.head(1)

,txNomeParlamentar,cpf,ideCadastro,nuCarteiraParlamentar,nuLegislatura,sgUF,sgPartido,codLegislatura,numSubCota,txtDescricao,numEspecificacaoSubCota,txtDescricaoEspecificacao,txtFornecedor,txtCNPJCPF,txtNumero,indTipoDocumento,datEmissao,vlrDocumento,vlrGlosa,vlrLiquido,numMes,numAno,numParcela,txtPassageiro,txtTrecho,numLote,numRessarcimento,datPagamentoRestituicao,vlrRestituicao,nuDeputadoId,ideDocumento,urlDocumento
0,LID.GOV-CD,NaN,NaN,NaN,2015,NaN,NaN,55,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,0,NaN,CRISTIANE FERREIRA EPP,037.139.030/0010-8,693,4,2019-01-16,111.3,0,111.3,1,2019,0,NaN,NaN,1554246,NaN,NaN,NaN,2812,6738121,https://www.camara.leg.br/cota-parlamentar/not...


#### Tratando os dados da CEAP

##### Padronizando colunas `txtCNPJCPF`

In [10]:
df_reembolso["txtCNPJCPF"]

0         037.139.030/0010-8
1         037.139.030/0010-8
2         037.139.030/0010-8
3         037.139.030/0010-8
4         037.139.030/0010-8
                 ...        
289448    020.128.620/0016-0
289449     007.227.727/00  -
289450    020.128.620/0016-0
289451    000.000.000/0000-6
289452    096.026.750/0018-5
Name: txtCNPJCPF, Length: 289453, dtype: object

In [11]:
df_reembolso["txtCNPJCPF"] = df_reembolso["txtCNPJCPF"].str.replace(r'\D', '', regex=True)

df_reembolso["txtCNPJCPF"]

0         03713903000108
1         03713903000108
2         03713903000108
3         03713903000108
4         03713903000108
               ...      
289448    02012862000160
289449       00722772700
289450    02012862000160
289451    00000000000006
289452    09602675000185
Name: txtCNPJCPF, Length: 289453, dtype: object

##### Padronizando CPFs nulos

In [12]:
filtro_cpf_nulo = df_reembolso.cpf.isna()

filtro_cpf_nulo

0          True
1          True
2          True
3          True
4          True
          ...  
289448    False
289449    False
289450    False
289451    False
289452    False
Name: cpf, Length: 289453, dtype: bool

In [13]:
df_reembolso.loc[filtro_cpf_nulo, "cpf"] = "-1"

In [14]:
df_reembolso.cpf

0                   -1
1                   -1
2                   -1
3                   -1
4                   -1
              ...     
289448    5461154387.0
289449    9300885731.0
289450    9300885731.0
289451    9300885731.0
289452    9300885731.0
Name: cpf, Length: 289453, dtype: object

##### Padronizando Partidos nulos

In [15]:
filtro_partido_null = df_reembolso.sgPartido.isna()

filtro_partido_null

0          True
1          True
2          True
3          True
4          True
          ...  
289448    False
289449    False
289450    False
289451    False
289452    False
Name: sgPartido, Length: 289453, dtype: bool

In [16]:
df_reembolso.loc[filtro_partido_null, "sgPartido"] = "-1"

In [17]:
df_reembolso.sgPartido

0                   -1
1                   -1
2                   -1
3                   -1
4                   -1
              ...     
289448    REPUBLICANOS
289449              PL
289450              PL
289451              PL
289452              PL
Name: sgPartido, Length: 289453, dtype: object

##### Padronizando data de emissão

In [18]:
df_reembolso["datEmissao"]

0        2019-01-16
1        2019-02-04
2        2019-02-25
3        2019-03-28
4        2019-05-07
            ...    
289448   2019-12-11
289449   2019-12-31
289450   2019-12-18
289451          NaT
289452   2020-01-31
Name: datEmissao, Length: 289453, dtype: datetime64[ns]

In [19]:
df_reembolso["datEmissao"] = pd.to_datetime(df_reembolso["datEmissao"], format="%Y-%m-%d")

df_reembolso["datEmissao"]

0        2019-01-16
1        2019-02-04
2        2019-02-25
3        2019-03-28
4        2019-05-07
            ...    
289448   2019-12-11
289449   2019-12-31
289450   2019-12-18
289451          NaT
289452   2020-01-31
Name: datEmissao, Length: 289453, dtype: datetime64[ns]

### Receita Federal

#### Baixando os dados da RF

In [20]:
import requests

url = "https://serenata-de-amor-data.nyc3.digitaloceanspaces.com/2019-11-19-companies.csv.xz"

# fazendo o download do conteúdo do arquivo
r = requests.get(url)

# abrindo um arquivo no seu computador
file = open('2019-11-19-companies.csv.xz', "wb")

# escrever o conteúdo
file.write(r.content)
file.close()

#### Lendo os dados da RF

In [20]:
! pip install lzma

ERROR: Could not find a version that satisfies the requirement lzma (from versions: none)
ERROR: No matching distribution found for lzma


In [21]:
DTYPE = {
    'cnpj': str
}

df_empresas = pd.read_csv(
    '2019-11-19-companies.csv.xz',
    compression='xz',
    dtype=DTYPE,
)

### Juntando os dados

In [22]:
# fizemos o merge da tabela df_reembolso com df_empresas.
# LEMBRANDO:
# df_reembolso é considerada a tabela da esquerda, então ela é a left.
# df_empresas é considerada a tabela da direita, então ela é a right.
df_final = df_reembolso.merge(
    df_empresas,
    how='left',
    left_on='txtCNPJCPF',
    right_on='cnpj'
)

In [23]:
df_final[['txtCNPJCPF', 'cnpj']]

,txtCNPJCPF,cnpj
0,03713903000108,03713903000108
1,03713903000108,03713903000108
2,03713903000108,03713903000108
3,03713903000108,03713903000108
4,03713903000108,03713903000108
...,...,...
479099,00722772700,NaN
479100,02012862000160,02012862000160
479101,02012862000160,02012862000160
479102,00000000000006,NaN


## Some o total da coluna `vlrLiquido` agrupando por `txtCNPJCPF` e `txtFornecedor`

In [24]:
# aqui nós configuramos para o pandas mostrar números decimais considerando somente
# 2 casas decimais depois do ponto
pd.options.display.float_format = '{:,.2f}'.format

In [25]:
# agrupamos os dados por txtCNPJCPF e txtFornecedor.
# somamos o vlrLiquido considerando o grupo resultante pelo groupby
# nós pegamos tbm só os 10 primeiros itens através do [:10]
df_reembolso.groupby(
    ['txtCNPJCPF', 'txtFornecedor']
)['vlrLiquido'].sum().sort_values().to_frame()[:10]

,,vlrLiquido
txtCNPJCPF,txtFornecedor,
30159008000166,PAPELARIA E BAZAR SANTA LUZIA,2.20
73830069000116,J.V. COMÉRCIO DE PETRÓLEO LTDA.,2.55
18572225000269,Concessionária das Rodovias Centrais do Brasil S/A,3.30
04699200000135,W P PARK ESTACIONAMENTO LTDA ME,3.60
02415408000150,CONCESSIONARIA DE RODOVIAS DO OESTE DE SAO PAULO - VIAOESTE S/A,3.80
25453685000152,BRUNA BANDEIRA FERRARI ME,4.00
61012704000179,PANIFICADORA MERCURIO EIRELI - EPP,4.10
04506629000169,NOVA ALIANÇA ATACADO LTDA - EPP 2,4.59
28019100000189,VIAPAULISTA S.A.,4.60


### Ordene os dados da consulta anterior de forma decrescente

In [26]:
df_reembolso.groupby(['txtCNPJCPF', 'txtFornecedor'])['vlrLiquido'].sum().sort_values(
    ascending=False
).to_frame()[:10]

,,vlrLiquido
txtCNPJCPF,txtFornecedor,
02012862000160,Cia Aérea - TAM,"24,922,866.52"
07575651000159,Cia Aérea - GOL,"22,007,041.30"
09296295000160,Cia Aérea - AZUL,"6,440,767.66"
07319323000191,PANTANAL VEÍCULOS LTDA,"2,181,274.52"
02012862000160,TAM LINHAS AEREAS S/A.,"1,172,126.78"
02575829000148,Cia Aérea - AVIANCA,"1,137,800.63"
23721017000133,"T2 COMUNICACAO, VIDEO E PRODUCOES EIRELI - EPP","1,097,334.00"
00000000000001,CELULAR FUNCIONAL,"872,853.43"
02558157000162,TELEFÔNICA BRASIL S.A. VIVO,"860,813.15"


In [31]:
# pegando dados do final de semana
df_reembolso["datEmissao"].dt.dayofweek < 5

0          True
1          True
2          True
3          True
4          True
          ...  
289448     True
289449     True
289450     True
289451    False
289452     True
Name: datEmissao, Length: 289453, dtype: bool

In [32]:
# usando a opção de normalizar o resultado do value_counts
# também ordenamos por ordem decrescente
df_reembolso.txtDescricao.value_counts(normalize=True).sort_values(ascending=False).to_frame().head(10)

,proportion
txtDescricao,
PASSAGEM AÉREA - RPA,0.28
COMBUSTÍVEIS E LUBRIFICANTES.,0.22
"SERVIÇO DE TÁXI, PEDÁGIO E ESTACIONAMENTO",0.11
MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE PARLAMENTAR,0.08
TELEFONIA,0.08
FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,0.06
SERVIÇOS POSTAIS,0.05
DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,0.04
LOCAÇÃO OU FRETAMENTO DE VEÍCULOS AUTOMOTORES,0.02


In [33]:
df_reembolso["vlrLiquido"].describe()

count   289,453.00
mean        730.24
std       2,083.63
min      -6,605.57
25%          50.00
50%         191.84
75%         715.75
max     145,140.00
Name: vlrLiquido, dtype: float64

In [34]:
# usamos 2 filtros aqui, para txtDescricao e para vlrLiquido

filtro_cat_alim = df_reembolso.txtDescricao == 'FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR'
filtro_vlr_max = df_reembolso.vlrLiquido > 5000

df_reembolso[(filtro_cat_alim) & (filtro_vlr_max)]

,txNomeParlamentar,cpf,ideCadastro,nuCarteiraParlamentar,nuLegislatura,sgUF,sgPartido,codLegislatura,numSubCota,txtDescricao,numEspecificacaoSubCota,txtDescricaoEspecificacao,txtFornecedor,txtCNPJCPF,txtNumero,indTipoDocumento,datEmissao,vlrDocumento,vlrGlosa,vlrLiquido,numMes,numAno,numParcela,txtPassageiro,txtTrecho,numLote,numRessarcimento,datPagamentoRestituicao,vlrRestituicao,nuDeputadoId,ideDocumento,urlDocumento
152,LIDERANÇA DO PARTIDO REPUBLICANO DA ORDEM SOCIAL,-1,NaN,NaN,2019,NaN,-1,56,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,0,NaN,ARTESANAL SERVICOS DE ALIMENTACAO E BUFFET EIRELI,22836210000157,7,4,2019-12-11 00:00:00,"6,000.00",0,"6,000.00",11,2019,0,NaN,NaN,1661111,NaN,NaN,NaN,2865,6981848,https://www.camara.leg.br/cota-parlamentar/not...
159,LIDERANÇA DO PARTIDO REPUBLICANO DA ORDEM SOCIAL,-1,NaN,NaN,2019,NaN,-1,56,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,0,NaN,SILT SELF SERVICE EIRELI,07286731000194,636,4,2019-11-05 23:00:00,"6,000.00",0,"6,000.00",10,2019,0,NaN,NaN,1650383,NaN,NaN,NaN,2865,6956990,https://www.camara.leg.br/cota-parlamentar/not...
160,LIDERANÇA DO PARTIDO REPUBLICANO DA ORDEM SOCIAL,-1,NaN,NaN,2019,NaN,-1,56,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,0,NaN,SILT SELF SERVICE EIRELI ME,07286731000194,295,4,2019-04-03 00:00:00,"6,000.00",0,"6,000.00",4,2019,0,NaN,NaN,1581569,NaN,NaN,NaN,2865,6798895,https://www.camara.leg.br/cota-parlamentar/not...
164,LIDERANÇA DO PARTIDO REPUBLICANO DA ORDEM SOCIAL,-1,NaN,NaN,2019,NaN,-1,56,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,0,NaN,SILT SELF SERVICE EIRELI ME,07286731000194,394,4,2019-07-02 00:00:00,"5,400.00",0,"5,400.00",6,2019,0,NaN,NaN,1609922,NaN,NaN,NaN,2865,6864494,https://www.camara.leg.br/cota-parlamentar/not...
168,LIDERANÇA DO PARTIDO REPUBLICANO DA ORDEM SOCIAL,-1,NaN,NaN,2019,NaN,-1,56,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,0,NaN,SILT SELF SERVICE EIRELI ME,07286731000194,520,4,2019-09-25 00:00:00,"6,000.00",0,"6,000.00",9,2019,0,NaN,NaN,1636965,NaN,NaN,NaN,2865,6926450,https://www.camara.leg.br/cota-parlamentar/not...
264,LIDERANÇA DO PROGRESSISTAS,-1,NaN,NaN,2019,NaN,-1,56,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,0,NaN,RONILDA FRANCISCA RIBEIRO DOS REIS,20869131000117,001525287,0,2019-11-07 23:00:00,"8,000.00",0,"8,000.00",10,2019,0,NaN,NaN,1651228,NaN,NaN,NaN,3461,6959055,https://www.camara.leg.br/cota-parlamentar/doc...
265,LIDERANÇA DO PROGRESSISTAS,-1,NaN,NaN,2019,NaN,-1,56,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,0,NaN,RONILDA FRANCISCA RIBEIRO DOS REIS,20869131000117,001546052,0,2019-11-24 23:00:00,"8,000.00",0,"8,000.00",11,2019,0,NaN,NaN,1655704,NaN,NaN,NaN,3461,6969841,https://www.camara.leg.br/cota-parlamentar/doc...
266,LIDERANÇA DO PROGRESSISTAS,-1,NaN,NaN,2019,NaN,-1,56,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,0,NaN,RONILDA FRANCISCA RIBEIRO DOS REIS,20869131000117,001579281,0,2019-12-15 23:00:00,"8,000.00",0,"8,000.00",12,2019,0,NaN,NaN,1663508,NaN,NaN,NaN,3461,6987099,https://www.camara.leg.br/cota-parlamentar/doc...
267,LIDERANÇA DO PSD,-1,NaN,NaN,2015,NaN,-1,55,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,0,NaN,ROBERTTH MOREIRA DE JESUS,30561574000108,001186205,0,2019-01-31 00:00:00,"5,355.00",0,"5,355.00",1,2019,0,NaN,NaN,1558264,NaN,NaN,NaN,2725,6747304,https://www.camara.leg.br/cota-parlamentar/doc...
283,LIDERANÇA DO PSD,-1,NaN,NaN,2019,NaN,-1,56,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,0,NaN,RAFAEL DOS SANTOS SILVA 72238208100,22836210000157,001376062,0,2019-07-12 00:00:00,"5,249.85",0,"5,249.85",7,2019,0,NaN,NaN,1616327,NaN,NaN,NaN,2725,6879754,https://www.camara.leg.br/cota-parlamentar/doc...


In [35]:
# aqui nós agrupamos por partido, UF e nome do parlamentar.
# Somamos o vlrLiquido considerando os grupos gerados pelo groupby
# ordenamos de forma decrescente

df_reembolso.groupby(
    ['sgPartido', 'sgUF', 'txNomeParlamentar']
)["vlrLiquido"].sum().sort_values(ascending=False).to_frame()

,,,vlrLiquido
sgPartido,sgUF,txNomeParlamentar,
PP,RR,Hiran Gonçalves,"542,487.76"
MDB,AC,Jéssica Sales,"537,322.05"
PDT,RS,Afonso Motta,"537,270.79"
PCdoB,AC,Perpétua Almeida,"536,138.75"
MDB,AC,Flaviano Melo,"520,071.45"
...,...,...,...
PCdoB,PE,LUCIANA SANTOS,0.49
PSL,MG,Marcelo Álvaro Antônio,-270.84
PODE,PR,Luiz Carlos Hauly,-784.47


### Consulte se houveram gastos reembolsados associados a empresas que na época estavam com status `08`


status == 8 significa `BAIXADA` (*a empresa foi extinta, seja por solicitação dos sócios ou por ação da Receita*)

In [36]:
# situation == 8 significa empresa que está BAIXADA
f = df_final.situation == 8

df_final[f].shape

(293, 75)

In [37]:
# datEmissao é a coluna que informa a data em que o gasto foi feito (vem da base da CEAP)
# situation_date é a coluna que informa a data em que a empresa ficou com a situação da coluna situation (vem da Receita Federal)
df_final["datEmissao"] = pd.to_datetime(df_final["datEmissao"], format="%Y-%m-%d")
df_final['situation_date'] = pd.to_datetime(df_final["situation_date"], format="%Y-%m-%d")

Lembrando, se temos `datEmissao` como `02/10/2020` e `situation_date` como `01/10/2020`, o python/pandas vai entender que `datEmissao` contém uma data posterior a `situation_date`, logo, é uma _data maior_. Por isso podemos fazer algo como `datEmissao` > `situation_date`, isso vai ser verdadeiro se a `datEmissao` for uma data que veio depois de `situation_date`.

In [38]:
# aqui vamos pesquisar se a data do gasto veio depois da data em que a empresa mudou de situação
# vamos agrupar por txNomeParlamentar
# aplicamos count em ideDocumento e sum em vlrLiquido
# ordenamos o resultado por vlrLiquido
(
    df_final[f].query('datEmissao > situation_date')
    .groupby('txNomeParlamentar')
    .agg({'ideDocumento': 'count', 'vlrLiquido': 'sum'})
    .sort_values(by='vlrLiquido', ascending=False)
    .reset_index()
)

,txNomeParlamentar,ideDocumento,vlrLiquido
0,Gleisi Hoffmann,42,"62,065.18"
1,Carlos Veras,10,"11,396.40"
2,Carla Zambelli,1,"7,289.65"
3,Darci de Matos,1,"2,869.38"
4,General Peternelli,22,"2,805.31"
5,Geovania de Sá,2,"2,703.30"
6,Lucas Redecker,8,"2,308.40"
7,Enéias Reis,1,"1,729.39"
8,Carlos Gomes,1,"1,560.00"
9,José Nunes,1,"1,510.40"


In [39]:
df_final.columns

Index(['txNomeParlamentar', 'cpf', 'ideCadastro', 'nuCarteiraParlamentar',
       'nuLegislatura', 'sgUF', 'sgPartido', 'codLegislatura', 'numSubCota',
       'txtDescricao', 'numEspecificacaoSubCota', 'txtDescricaoEspecificacao',
       'txtFornecedor', 'txtCNPJCPF', 'txtNumero', 'indTipoDocumento',
       'datEmissao', 'vlrDocumento', 'vlrGlosa', 'vlrLiquido', 'numMes',
       'numAno', 'numParcela', 'txtPassageiro', 'txtTrecho', 'numLote',
       'numRessarcimento', 'datPagamentoRestituicao', 'vlrRestituicao',
       'nuDeputadoId', 'ideDocumento', 'urlDocumento', 'cnpj', 'partners',
       'secondary_activities', 'latitude', 'longitude', 'opening',
       'main_activity', 'neighborhood', 'share_capital', 'zip_code',
       'main_activity_code', 'city_code', 'judicial_nature_code',
       'additional_address_details', 'removed_from_simples_since', 'opened_in',
       'opted_for_comples_in', 'situation_date', 'special_situation_date',
       'fax_area_code', 'phone1_area_code', 'phon

In [40]:
(
    df_final[f].query('datEmissao > situation_date')
    .groupby(['txNomeParlamentar', 'txtFornecedor', 'situation'])['ideCadastro'].count()
    .sort_values(ascending=False).to_frame()
)

,,,ideCadastro
txNomeParlamentar,txtFornecedor,situation,
Gleisi Hoffmann,GOL LINHAS AÉREAS INTELIGENTES,8.00,42
General Peternelli,PREFEITURA DE SÃO PAULO,8.00,18
Carlos Veras,GOL LINHAS AÉREAS INTELIGENTES,8.00,10
Lucas Redecker,NET SERVICOS DE COMUNICACAO S/A,8.00,8
Professora Dayane Pimentel,NET,8.00,5
Capitão Augusto,TRANSBRASILIANA - CONCESSIONARIA DE RODOVIA S.A.,8.00,5
General Peternelli,HOTELARIA ACCOR BRASIL S/A,8.00,4
Professora Dayane Pimentel,Net Serviços de Comunicação S/A,8.00,4
José Airton Félix Cirilo,TIM Celular SA,8.00,4
